In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

from keras.models import Sequential
from keras.layers import (Conv2D, GlobalAveragePooling2D, BatchNormalization, Flatten,
                          GlobalMaxPool2D, MaxPool2D, concatenate, Activation, Input, Dense)
import keras.optimizers
import matplotlib.pyplot as plt
from keras.utils import plot_model
from keras.utils import np_utils
from keras.preprocessing import sequence
from keras import optimizers, metrics, regularizers

Using TensorFlow backend.


In [2]:
train_dataX = np.load('train_data.npy')
test_dataX = np.load('test_data.npy')
train_datay = pd.read_csv('train_labels.csv')

#pre-processing data
train_datay = train_datay.drop('Id',1)
train_dataX = train_dataX.reshape(-1,210,210,1)
test_dataX = test_dataX.reshape(-1,210,210,1)
Y_train = np_utils.to_categorical(train_datay,2)

#splitting data for validation
X_train, X_test, y_train, y_test = train_test_split(train_dataX,Y_train, test_size=0.10, shuffle=True)

#deleting unwanted variables
del train_dataX, train_datay

In [3]:
model = Sequential()
model.add(Conv2D(64, (3,6),input_shape=(210,210,1)))
model.add(BatchNormalization())
model.add(Activation("tanh"))
model.add(MaxPool2D())

model.add(Conv2D(32, (3,6)))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(MaxPool2D())

model.add(Conv2D(32, (3,6)))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(MaxPool2D())

model.add(Flatten())
model.add(Dense(2,activation='softmax'))
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 208, 205, 64)      1216      
_________________________________________________________________
batch_normalization_1 (Batch (None, 208, 205, 64)      256       
_________________________________________________________________
activation_1 (Activation)    (None, 208, 205, 64)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 104, 102, 64)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 102, 97, 32)       36896     
_________________________________________________________________
batch_normalization_2 (Batch (None, 102, 97, 32)       128       
_________________________________________________________________
acti

In [4]:
# compile model
sgd=optimizers.SGD(lr=0.01)
adam=optimizers.Adam(lr=0.01)
model.compile(loss='categorical_crossentropy', 
              optimizer=adam,
              metrics=['accuracy'])

In [5]:
model.fit(X_train, y_train,
          batch_size=100, nb_epoch=10,verbose=1,
          validation_data=(X_test, y_test))

Instructions for updating:
Use tf.cast instead.


d:\programs\python installed\lib\site-packages\ipykernel_launcher.py:3: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  This is separate from the ipykernel package so we can avoid doing imports until


Train on 3599 samples, validate on 400 samples
Epoch 1/10


ResourceExhaustedError: OOM when allocating tensor with shape[100,64,208,205] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node training/Adam/gradients/max_pooling2d_1/MaxPool_grad/MaxPoolGrad}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


In [ ]:
from sklearn.metrics import roc_auc_score

Y_test_predClass = model.predict_proba(X_valid)


roc = roc_auc_score(y_valid, Y_test_predClass)
print("ROC:" + str(round(roc,4)))


In [ ]:
Y_test_pred = model.predict(test_dataX)
print(Y_test_pred)

In [ ]:
pd.DataFrame(Y_test_pred[:,0]).to_csv('submission52.csv', header=["Label"], index_label="Id")
del model